# Set Up Scheduler Configuration

In [ ]:
import sys
sys.path.append('/home/b/bechtol/repos/ts_config_ocs')

In [ ]:
import Scheduler.feature_scheduler.maintel.fbs_config_fieldsurvey

In [ ]:
nside, scheduler = Scheduler.feature_scheduler.maintel.fbs_config_fieldsurvey.get_scheduler()

In [ ]:
scheduler.survey_lists

In [ ]:
scheduler.survey_lists[0][0].basis_functions

In [ ]:
scheduler.survey_lists[0][0].detailers

# Set Up Model Observatory

In [ ]:
import os
import warnings
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import colorcet as cc
import healpy as hp
from astropy.time import Time

from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site
from rubin_scheduler.data import get_data_dir

from rubin_scheduler.scheduler.model_observatory import ModelObservatory, KinemModel
from rubin_scheduler.site_models import ConstantSeeingData, ConstantWindData

from rubin_scheduler.utils import ra_dec2_hpid
from rubin_scheduler.scheduler.utils import empty_observation

from rubin_scheduler.scheduler import basis_functions
from rubin_scheduler.scheduler import features
from rubin_scheduler.scheduler.detailers import CameraSmallRotPerObservationListDetailer
from rubin_scheduler.scheduler.surveys import FieldSurvey
from rubin_scheduler.scheduler.schedulers import CoreScheduler, ComCamFilterSched
from rubin_scheduler.utils import Site, _approx_altaz2pa, pseudo_parallactic_angle, rotation_converter

from rubin_scheduler.scheduler import sim_runner
from rubin_scheduler.scheduler.utils import SchemaConverter

In [ ]:
at_usdf = True
if at_usdf:
    #os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/group/rubin/shared/rubin_sim_data"
    os.environ["RUBIN_SIM_DATA_DIR"] = "/home/b/bechtol/rubin-user/rubin_sim_data"
print("Using rubin_sim_data from ", get_data_dir())

In [ ]:
# Setting the start of the *survey* (and keeping this the same) is important for the Model Observatory and Scheduler,
# because this sets an overall dither pattern per night, but also helps track things that may 
# otherwise change per night ... for SV surveys, might not be totally necessary, but is good practice
# (you can change the *day* / mjd that you are simulating, of course, but mjd_start should remain the same)

dayobs = '2024-09-09'
survey_start = Time(f'{dayobs}T12:00:00', format='isot', scale='utc')

In [ ]:
# Don't have to do this, but can grab almanac information

almanac = Almanac(mjd_start = survey_start.mjd)

def show_almanac_info(dayobs):
    night_info = almanac.get_sunset_info(evening_date=dayobs, longitude=Site('LSST').longitude_rad)
    
    dd = []
    for val, col in zip(night_info, night_info.dtype.names):
        if col == 'night':
            continue
        else:
            print(col, val, Time(val, format='mjd', scale='utc').iso)
    
    # And can check on the lunar phase -- this goes from 0 (new) to 100 (full)
    moon_phase = almanac.get_sun_moon_positions(night_info['moonrise'])['moon_phase']
    print(f'moonphase(%) {moon_phase.round(2)}')

show_almanac_info(dayobs)

In [ ]:
def tma_movement(percent=70):
    # See https://confluence.lsstcorp.org/display/LSSTCOM/TMA+Motion+Settings
    # Expected performance at end of comcam on-sky is probably 10%
    if percent > 125:
        percent = 125
        print("Cannot exceed 125 percent, by requirements.")
    tma = {}
    scale = percent / 100.0
    tma['azimuth_maxspeed'] = np.min([10.0 * scale, 7.0])
    tma['azimuth_accel'] = 10.0 * scale
    tma['azimuth_jerk'] = np.max([1.0, 40.0 * scale])
    tma['altitude_maxspeed'] = 5.0 * scale
    tma['altitude_accel'] = 5.0 * scale
    tma['altitude_jerk'] = np.max([1.0, 20.0 * scale])
    tma['settle_time'] = 3.0
    return tma

In [ ]:
def rotator_movement(percent=100):
    # Kevin and Brian say these can run 100% and are independent of TMA movement
    if percent > 125:
        percent = 125
        print("Cannot exceed 125 percent, by requirements.")
    rot = {}
    rot['maxspeed'] = 3.5 * percent/100
    rot['accel'] = 1.0 * percent/100
    rot['jerk'] = 4.0 * percent/100
    return rot

In [ ]:
def get_model_observatory(dayobs='2024-09-09'):
    # Set up a fresh model observatory
    survey_start = Time(f'{dayobs}T12:00:00', format='isot', scale='utc')
    mjd_now = survey_start.mjd

    kinematic_model = KinemModel(mjd0=mjd_now)
    kinematic_model.setup_camera(readtime=2.4, **rotator_movement(100.))
    kinematic_model.setup_telescope(**tma_movement(10.0))
    
    # Some weather telemetry that might be useful
    seeing_data = ConstantSeeingData(fwhm_500=2.0)
    wind_data = ConstantWindData(wind_direction=340, wind_speed=5.0)
    
    # Set up the model observatory
    observatory = ModelObservatory(mjd=mjd_now, 
                                   mjd_start=survey_start.mjd,
                                   kinem_model=kinematic_model, # Modified kinematics
                                   cloud_data='ideal',          # No clouds
                                   seeing_data=seeing_data,     # Modified seeing
                                   wind_data=wind_data,         # Add some wind
                                   downtimes='ideal',           # No downtime
                                   lax_dome=True,               # dome crawl?
                                   init_load_length=1,          # size of skybrightness files to load first
                                  )
    return observatory

In [ ]:
def update_model_observatory_sunset(observatory, filter_scheduler):
    # Make sure correct filters are mounted
    conditions = observatory.return_conditions()
    filters_needed = filter_scheduler(conditions)
    observatory.observatory.mount_filters(filters_needed)
    conditions = observatory.return_conditions()
    print("Current filter set", conditions.mounted_filters)
    
    # Move ahead to twilight
    observatory.mjd = conditions.sun_n18_setting
    print("Current observatory time", observatory.mjd)
    return observatory

# Run Simulation

In [ ]:
verbose = False

dayobs = '2024-09-09'
survey_start = Time(f'{dayobs}T12:00:00', format='isot', scale='utc')
mjd_now = survey_start.mjd

# Set up again to be sure all is 'clean'
#scheduler, filter_scheduler = get_comcam_schedulers()
filter_scheduler = ComCamFilterSched()
observatory = get_model_observatory(dayobs)
observatory = update_model_observatory_sunset(observatory, filter_scheduler)
conditions = observatory.return_conditions()

mjd = observatory.mjd
mjd_end = conditions.sun_n12_rising
survey_length = 1 #mjd_end - mjd

observatory, scheduler, observations = sim_runner(
    observatory,
    scheduler,
    filter_scheduler=filter_scheduler,
    mjd_start=mjd,
    survey_length=survey_length,
    n_visit_limit=None,
    step_none=10.0,
    verbose=True,
    record_rewards=False,
    start_result_size=int(2e4),
    append_result_size=int(2.5e4),
)

In [ ]:
sc = SchemaConverter()
visits = sc.obs2opsim(observations)

In [ ]:
# Let's check in on sequences, because there are some issues
visits.groupby('target')['observationStartMJD'].count()

In [ ]:
# the time from the start of one exposure to the start of the next exposure:
slew_to_next_visit = visits['slewTime'][1:]
expected_overheads = visits['visitTime'][:-1].values + slew_to_next_visit.values
# Compare to the actual time between visits
actual_overheads = np.diff(visits['observationStartMJD'])*24*60*60 
# We can look at the difference to identify gaps in visits (seconds)
diff = np.abs(expected_overheads - actual_overheads)
indx = np.where((diff > 1) & (diff < 6*60*60))[0]
breaks = []
for i in indx:
    b1 = visits.iloc[i]['observationStartMJD']
    b2 = visits.iloc[i+1]['observationStartMJD']
    breaks.append([b1, b2])
print(indx)
print(breaks)

In [ ]:
plt.figure(figsize=(8, 6))
filtershapes = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}
filtercolors = {'u': '#005BDB',
                'g': '#3DA952',
                'r': '#c61c00',
                'i': '#997500',
                'z': '#CE0D77',
                'y': '#5d0000'}
fieldcolors = {}
for i, t in enumerate(visits.target.unique()):
    fieldcolors[t] = cc.glasbey[i]

if len(breaks) > 0:
    for b in breaks:
        plt.fill_between(b, 2.5, 0.0, color='pink', alpha=0.3)

night = 1
nvisits = visits.query('night == @night')
for t in nvisits.target.unique():
    vv = nvisits.query('target == @t')
    for f in vv['filter'].unique():
        vvf = vv.query('filter == @f')
        label = f'{t} {f}'
        plt.plot(vvf.observationStartMJD, vvf.airmass, linestyle='', 
                 marker=filtershapes[f], markersize=3, color=fieldcolors[t], label=label)
plt.legend(loc=(0.00 , -0.3), ncols=len(nvisits.target.unique()))
plt.ylim(2.5, 0.9)
plt.xlim(nvisits.observationStartMJD.min(), nvisits.observationStartMJD.max())
plt.grid(alpha=0.3)
plt.xlabel("MJD", fontsize='large')
plt.ylabel("Airmass", fontsize='x-large')
#plt.title(f"Night {night}, {Time(nights.observationStartMJD.min(), format='mjd', scale='utc').isot.split("T")[0]}")

# Scratch

In [ ]:
import lsst.ts.fbs.utils
lsst.ts.fbs.utils

In [ ]:
#import lsst.ts.config.ocs

In [ ]:
import rubin_scheduler
rubin_scheduler

In [ ]:
from lsst.ts.fbs.utils.maintel import MakeFieldSurveyScheduler

In [ ]:
nside = 32

In [ ]:
make_scheduler = MakeFieldSurveyScheduler()

In [ ]:
make_scheduler.nside

In [ ]:
sequence="rgi"
nvis=[30, 30, 30]
survey_name="test"

In [ ]:
make_scheduler._load_candidate_fields()

In [ ]:
field_names = [
    "EDFS_A", 
    "EDFS_B",
]
make_scheduler.add_field_surveys(field_names, sequence=sequence, nvis=nvis, survey_name=survey_name)

In [ ]:
make_scheduler.surveys[1]

In [ ]:
make_scheduler.get_scheduler()

# Scheduler Config Testing

In [ ]:
import sys
sys.path.append('/home/b/bechtol/repos/ts_config_ocs')

In [ ]:
#import Scheduler.feature_scheduler.auxtel.fbs_config_image_photocal_survey

In [ ]:
#Scheduler.feature_scheduler.auxtel.fbs_config_image_photocal_survey.get_scheduler()

In [ ]:
import Scheduler.feature_scheduler.maintel.fbs_config_fieldsurvey

In [ ]:
nside, scheduler = Scheduler.feature_scheduler.maintel.fbs_config_fieldsurvey.get_scheduler()

In [ ]:
scheduler.survey_lists

In [ ]:
dir(scheduler.survey_lists[0][0])

In [ ]:
scheduler.survey_lists[0][0].basis_functions

In [ ]:
scheduler.survey_lists[0][0].detailers

# Run Simulation Simple

In [ ]:
import copy
import numpy as np

from astropy.time import Time

import matplotlib.pylab as plt

In [ ]:
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.site_models import SeeingData
from rubin_scheduler.scheduler import  sim_runner

In [ ]:
# There ought to be a way to set the environment variable for this kernel
import os
os.environ["RUBIN_SIM_DATA_DIR"] = "/home/b/bechtol/rubin-user/rubin_sim_data"
os.getenv("RUBIN_SIM_DATA_DIR")

In [ ]:
# Note that can only set this date after downloading the full set sky brightness data
mjd_start = Time('2024-09-01 00:00:00.000', format='iso').mjd
mjd_start

In [ ]:
# We need the start date of the survey, so let's load up our model observatory and get that from the conditions
mo = ModelObservatory(nside=nside, ideal_conditions=True, mjd_start=mjd_start)
mo.seeing_data = SeeingData(Time(mjd_start, format='mjd'))
conditions = mo.return_conditions()

In [ ]:
conditions.mounted_filters = ['gjgj', 'ghghi', 'kykykz']
conditions.mounted_filters

In [ ]:
mo.conditions.mounted_filters

In [ ]:
new_mo, new_scheduler, observations = sim_runner(copy.deepcopy(mo), copy.deepcopy(scheduler), survey_length=0.2, verbose=True)
del new_mo
del new_scheduler

In [ ]:
observations.dtype.names

In [ ]:
observations['filter']

In [ ]:
observations[0]

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], observations['airmass'][in_filt], 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()
plt.xlabel('MJD')
plt.ylabel('Airmass')